# Store-Item Sales Forecasting

In [ ]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [ ]:
from time import time

import altair as alt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
%aimport src.altair_helpers
from src.altair_helpers import altair_datetime_heatmap_grid, ts_plot

%aimport src.cv_helpers
from src.cv_helpers import MultiTimeSeriesDateSplit

%aimport src.data_custom_transformers
import src.data_custom_transformers as ct

%aimport src.data_helpers
from src.data_helpers import az_load_data

%aimport src.inference_helpers
from src.inference_helpers import predict_future

%aimport src.ml_helpers
import src.ml_helpers as mlh

%aimport src.ml_metrics
from src.ml_metrics import lgbm_smape, lgbm_smape_sklearn

%aimport src.utils
from src.utils import score_model

%aimport src.visualization_helpers
from src.visualization_helpers import plot_store_item_grid, show_splits

In [ ]:
pd.set_option("display.max_rows", 150)

<a href="table-of-contents"></a>

## [Table of Contents](#table-of-contents)
0. [About](#about)
1. [User Inputs](#user-inputs)
2. [Load Data](#load-data)
   - 2.1. [Read Data](#read-data)
   - 2.2. [Sort Data](#read-data)
   - 2.3. [Create Usable Data by Dropping Observations in Trailing Gap](#create-usable-data-by-dropping-observations-in-trailing-gap)
   - 2.4. [Create Overall Train-Test Splits, separated by a gap, from Usable Data](#create-overall-train-test-splits-,-separated-by-a-gap,-from-usable-data)
   - 2.5. [Set Multi-Index to Support CV Splitter](#set-multi-index-to-support-cv-splitter)
   - 2.6. [Exploratory Data Analysis](#exploratory-data-analysis)
     - 2.6.1. [Visualize Historical Sales for 10 store-item combos](#visualize-historical-sales-for-10-store-item-combos)
     - 2.6.2. [Check Normality](#check-normality)
     - 2.6.3. [TimeSeries Plots](#timeseries-plots)
3. [Cross Validation](#cross-validation)
   - 3.1. [Inspect Cross-Validation Folds](#inspect-cross-validation-folds)
   - 3.2. [Perform Cross-Validation](#perform-cross-validation)
   - 3.3. [Get best Model Hyper-parameters from Cross-Validation](#get-best-model-hyper-parameters-from-cross-validation)
4. [Model Evaluation](#model-evaluation)
   - 4.1. [Train on Overall Training Split, Skip over Gap, Predict Overall Testing Split](#train-on-overall-training-split,-skip-over-gap,-predict-overall-testing-split)
   - 4.2. [Retrieve True and Predicted Values of Overall Testing Split](#retrieve-true-and-predicted-values-of-overall-testing-split)
   - 4.3. [Model Assessment on Overall Testing Split](#model-assessment-on-overall-testing-split)
5. [Predict into the Future](#predict-into-the-future)
   - 5.1. [Read in all Data](#read-in-all-data)
   - 5.2. [Sort all Data](#sort-all-data)
   - 5.3. [Create Usable Data by Dropping Observations in Trailing Gap from all Data](#create-usable-data-by-dropping-observations-in-trailing-gap-from-all-data)
   - 5.4. [Set Multi-Index to Support CV Splitter for all Data](#set-multi-index-to-support-cv-splitter-for-all-data)
   - 5.5. [Make Predictions](#make-predictions)
6. [Notes](#notes)

<a id="about"></a>

## 0. [About](#about)

Here, Machine Learning (ML) will be used to generate sales forecasts (quantity) for each of 50 items being sold in 10 separate stores.

<a id="user-inputs"></a>

## 1. [User Inputs](#user-inputs)

In [ ]:
# ts_plot inputs
plot_xvar = "date"
plot_yvar = "sales"
roll_window = 365
hist_bin_step_size = 5
n_lags = 50
alpha = 0.05
ci_band_opacity = 0.5
fig_size = [370, 275]
corr_plots_wanted = ["acf", "pacf"]

# Heatmap inputs
heatmap_agg = "median"

# ML Inputs 1/2
N_SPLITS = 5
HORIZON = 90
GAP = 90

# ML Inputs 2/2
lags_range = [90, 120]
window_size = 180
used_columns = [
    "store",
    "item",
    "date",
    "sales",
]
categ_fea = ["store", "item"]

In [ ]:
az_storage_container_name = "myconedesx7"
blob_dict_inputs = {
    "blobedesz36": "train",
    "blobedesz37": "test",
}

fig_size = tuple(fig_size)
ts_tooltip = [
    alt.Tooltip(f"{plot_xvar}:T", title="Date"),
    alt.Tooltip(f"{plot_yvar}:Q", title="Sales"),
]
residual_tooltip = [
    alt.Tooltip(f"{plot_xvar}:T", title="Date"),
    alt.Tooltip(f"{plot_yvar}:Q", title="Sales", format=".1f"),
]
corr_plots_tooltip = {}
for auto_corr_type in ["acf", "pacf"]:
    plot_tooltip = [
        alt.Tooltip(f"lag:Q", title="lag"),
        alt.Tooltip(f"high_ci - low_ci:Q", title="CI (Upper-Lower)/2", format=".2f"),
        alt.Tooltip(f"low_ci:Q", title="CI Lower bound", format=".2f"),
        alt.Tooltip(f"{auto_corr_type}:Q", title=auto_corr_type.upper(), format=".2f"),
        alt.Tooltip(f"high_ci:Q", title="CI Upper bound", format=".2f"),
    ]
    corr_plots_tooltip[auto_corr_type] = plot_tooltip

lags = np.arange(lags_range[0], lags_range[1])

test_period_length = HORIZON
lookahead = GAP + 1

<a id="load-data"></a>

## 2. [Load Data](#load-data)

<a id="read-data"></a>

### 2.1. [Read Data](#read-data)

In [ ]:
%%time
data = az_load_data(
    blob_dict_inputs,
    az_storage_container_name,
    ["date"],
    ["train"],
)[0]
print(f"Train data shape = {data.shape[0]}")
display(data.head().append(data.tail()))
display(data.dtypes.to_frame())

<a id="sort-data"></a>

### 2.2. [Sort Data](#read-data)

In [ ]:
%%time
multi_col_sort_pipe = Pipeline(
    [
        ("sisort", ct.DFMultiColSort(["store", "item", "date"], [True, True, True])),
    ]
)
data = multi_col_sort_pipe.fit_transform(data)
display(data)

In [ ]:
FIRST_DATE = data["date"].min()
FIRST_DATE

<a id="create-usable-data-by-dropping-observations-in-trailing-gap"></a>

### 2.3. [Create Usable Data by Dropping Observations in Trailing Gap](#create-usable-data-by-dropping-observations-in-trailing-gap)

In [ ]:
train_end_date = data["date"].max() - pd.DateOffset(days=GAP)
gap_dates = pd.date_range(train_end_date + pd.DateOffset(days=1), data["date"].max())
print(len(gap_dates))
data = data[data["date"] <= train_end_date]
display(data.head().append(data.tail()))

<a id="create-overall-train-test-splits-,-separated-by-a-gap,-from-usable-data"></a>

### 2.4. [Create Overall Train-Test Splits, separated by a gap, from Usable Data](#create-overall-train-test-splits-,-separated-by-a-gap,-from-usable-data)

In [ ]:
train_split_end_date = data["date"].max() - pd.DateOffset(days=GAP * 2)
gap_dates = pd.date_range(
    train_split_end_date + pd.DateOffset(days=1), data["date"].max()
)
data_train = data[data["date"] <= train_split_end_date]
display(data_train.head().append(data_train.tail()))

<a id="set-multi-index-to-support-cv-splitter"></a>

### 2.5. [Set Multi-Index to Support CV Splitter](#set-multi-index-to-support-cv-splitter)

In [ ]:
%%time
data = data.assign(
    symbol=data["store"].astype(str) + "_" + data["item"].astype(str)
).set_index(["symbol", "date"])
display(data)

In [ ]:
%%time
data_train = data_train.assign(
    symbol=data_train["store"].astype(str) + "_" + data_train["item"].astype(str)
).set_index(["symbol", "date"])
display(data_train)

<a id="exploratory-data-analysis"></a>

### 2.6. [Exploratory Data Analysis](#exploratory-data-analysis)

<a id="visualize-historical-sales-for-10-store-item-combos"></a>

#### 2.6.1. [Visualize Historical Sales for 10 store-item combos](#visualize-historical-sales-for-10-store-item-combos)

Visualize data for random store-item combinations

In [ ]:
%%time
store_item_tsplot_dict = {
    "store": np.random.randint(1, 10 + 1, size=10),
    "item": np.random.randint(1, 50 + 1, size=10),
}
plot_store_item_grid(
    data.reset_index(level=1), store_item_tsplot_dict, "2015", "2016", "sales", fig_size=(12, 20)
)

<a id="check-normality"></a>

#### 2.6.2. [Check Normality](#check-normality)

Check normality ([1](https://en.wikipedia.org/wiki/Normality_test), [2](https://en.wikipedia.org/wiki/Normal_distribution), [3](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.normaltest.html#scipy.stats.normaltest)) of `sales` variable for randomly generated (above) store-item combinations

In [ ]:
%%time
for store, item in zip(
    store_item_tsplot_dict["store"],
    store_item_tsplot_dict["item"],
):
    df_si = data_train[(data_train["store"]==store) & (data_train["item"]==item)]
    is_normal = mlh.check_normality(df_si["sales"])
    print(
        f"Are sales for store={store} and item={item} normally distributed "
        f"at 99% threshold? {is_normal}"
    )

Check normality of `sales` variable for all store-item combinations

In [ ]:
data_train_normal = data_train.merge(
    data_train.groupby(["store", "item"])["sales"]
    .apply(mlh.check_normality)
    .rename("is_normal")
    .to_frame()
    .reset_index(),
    left_on=["store", "item"],
    right_on=["store", "item"],
)
display(
    data_train_normal[["store", "item", "is_normal"]]
    .head()
    .append(data_train_normal[["store", "item", "is_normal"]].tail())
)
print(
    f"Number of normally distributed timeseries = {len(data_train_normal.loc[~data_train_normal['is_normal']])}"
)

<a id="timeseries-plots"></a>

#### 2.6.3. [TimeSeries Plots](#timeseries-plots)

Visualize datetime attributes for single store-item combination

In [ ]:
store, item = store_item_tsplot_dict["store"][0], store_item_tsplot_dict["item"][0]
train_small = data_train.reset_index(level=1)[
    (data_train.reset_index(level=1)["store"] == store)
    & (data_train.reset_index(level=1)["item"] == item)
]
heatmap_grid = altair_datetime_heatmap_grid(
    train_small,
    date_col="date",
    yvar="sales",
    ptitle=f"{heatmap_agg.title()} Sales (store={store}, item={item})",
    marker_linewidth=0.2,
    agg=heatmap_agg,
    ptitle_x_loc=15,
    chart_separation=5,
    fig_width=245,
    fig_half_heights=(350, 370),
    fig_height=750,
    cmap="yelloworangered",
    scale="linear",
)
ts_eda_plot = ts_plot(
    df=train_small[["date", "sales"]],
    xvar=plot_xvar,
    yvar=plot_yvar,
    ts_tooltip=ts_tooltip,
    corr_plots_tooltip=corr_plots_tooltip,
    line_roll_window=roll_window,
    line_linewidth=1.5,
    line_roll_stat_linewidth=3,
    line_roll_stat_colors=["red", "black"],
    hist_bin=alt.Bin(step=hist_bin_step_size),
    hist_bar_line_thickness=2,
    hist_bar_line_color="blue",
    hist_line_thickness=2,
    n_lags=n_lags,
    alpha=alpha,
    ci_band_opacity=ci_band_opacity,
    corr_plots_wanted=corr_plots_wanted,
    corr_plots_marker_size=40,
    corr_plots_vertical_line_width=1.5,
    corr_plots_marker_linewidth=1.5,
    corr_plots_zero_y_linewidth=1.5,
    axis_thickness=2,
    line_plot_fig_size=fig_size,
    hist_plot_fig_size=fig_size,
    corr_plots_fig_size=fig_size,
)
display(heatmap_grid, ts_eda_plot)

<a id="cross-validation"></a>

## 3. [Cross Validation](#cross-validation)

<a id="inspect-cross-validation-folds"></a>

### 3.1. [Inspect Cross-Validation Folds](#inspect-cross-validation-folds)

In [ ]:
cv = MultiTimeSeriesDateSplit(
    num_folds=3,
    forecast_horizon=test_period_length,
    look_ahead_length=lookahead,  # GAP+1
)
ts_name = f"{store}_{item}"
df_cv = (
    data_train.reset_index()
    .sort_values(by=["symbol", "date"])
    .set_index(["symbol", "date"])
)
n_splits = cv.get_n_splits(data_train[categ_fea], data_train["sales"])
fig, ax = plt.subplots(figsize=(12, 4))
for fold_number, (train_idx, test_idx) in enumerate(cv.split(X=df_cv)):
    train_cv, test_cv = df_cv.iloc[train_idx], df_cv.iloc[test_idx]
    show_splits(
        train_cv,
        test_cv,
        fold_number + 1,
        ts_name,
        n_splits,
        "Expanding Window",
        ax,
        lw=5,
    )

In [ ]:
# Training API
lgb_params_all = [
    {
        "metric": {"mae"},
        "num_leaves": 10,
        "learning_rate": 0.02,
        "feature_fraction": 0.8,
        "max_depth": 5,
        "verbose": 0,
        "nthread": -1,
    }
]
lgb_fit_params = {
    "num_boost_round": 800,  # 2_200
    "early_stopping_rounds": 100,  # 200
}
# # sklearn API
# lgb_params_all = {
#     # "metric": {"mae"},  # not available?
#     "num_leaves": 10,
#     "learning_rate": 0.02,
#     "colsample_bytree": 0.8,  # feature_fraction = colsample_bytree
#     "max_depth": 5,
#     # "verbose": 0,  # .fit()
#     "n_estimators": 2_200,
#     # "early_stopping_rounds": 200,  # .fit()
#     "n_jobs": -1,
# }
# lgb_fit_params = {
#     "early_stopping_rounds": 200,
#     "verbose": 200,
# }

<a id="perform-cross-validation"></a>

### 3.2. [Perform Cross-Validation](#perform-cross-validation)

In [ ]:
%%time
df_cv_summaries = [
    pd.DataFrame.from_records(
        score_model(
            data_train,
            cv,
            lags,
            window_size,
            used_columns,
            categ_fea,
            FIRST_DATE,
            GAP,
            HORIZON,
            lgb_params,
            lgbm_smape,  # use lgbm_smape or lgbm_smape_sklearn
            lgb_fit_params,
        )
    )
    for lgb_params in lgb_params_all
]
df_cv_summary = pd.concat(df_cv_summaries)
display(df_cv_summary)

<a id="get-best-model-hyper-parameters-from-cross-validation"></a>

### 3.3. [Get best Model Hyper-parameters from Cross-Validation](#get-best-model-hyper-parameters-from-cross-validation)

In [ ]:
best_lgb_params = mlh.get_best_model_hyper_params(df_cv_summary, "smape")
best_lgb_params

<a id="model-evaluation"></a>

## 4. [Model Evaluation](#model-evaluation)

The ML model will now be instantiated with the best hyper-parameters found during hyper-parameter tuning (using cross-validation) above, and then trained on the overall training split and used to predict the overall testing split

<a id="train-on-overall-training-split,-skip-over-gap,-predict-overall-testing-split"></a>

### 4.1. [Train on Overall Training Split, Skip over Gap, Predict Overall Testing Split](#train-on-overall-training-split,-skip-over-gap,-predict-overall-testing-split)

With `cv=1`, the usable data created earlier in the subsection **Create Usable Data by Dropping Observations in Trailing Gap** will be re-divided into train and test splits (separated by a gap)
- the resulting train and test splits will be identical to the overall training and testing splits (`data_train` and `data_test` respectively) that were created earlier in the subsection **Create Overall Train-Test Splits (separated by Gap) from Usable Data**

In [ ]:
cv = MultiTimeSeriesDateSplit(
    num_folds=1,
    forecast_horizon=test_period_length,
    look_ahead_length=lookahead,  # GAP+1
)

In [ ]:
train_idx, test_idx = list(cv.split(data))[0]
data_train_redivided = data.iloc[train_idx]
data_test_redivided = data.iloc[test_idx]
assert data_train_redivided.equals(data_train)

Use LGBM with the best hyper-parameters found using hyper-parameter tuning to predict the overall testing split

In [ ]:
%%time
scoring_records_summary = score_model(
    data,
    cv,
    lags,
    window_size,
    used_columns,
    categ_fea,
    FIRST_DATE,
    GAP,
    HORIZON,
    best_lgb_params,
    lgbm_smape,  # use lgbm_smape or lgbm_smape_sklearn
    lgb_fit_params,
)

<a id="retrieve-true-and-predicted-values-of-overall-testing-split"></a>

### 4.2. [Retrieve True and Predicted Values of Overall Testing Split](#retrieve-true-and-predicted-values-of-overall-testing-split)

In [ ]:
y_true = data_test_redivided
y_pred = scoring_records_summary[0]["y_pred"]
assert y_true.index.equals(y_pred.index)

<a id="model-assessment-on-overall-testing-split"></a>

### 4.3. [Model Assessment on Overall Testing Split](#model-assessment-on-overall-testing-split)

**<span style='color:red'>To be done.</span>**

<a id="predict-into-the-future"></a>

## 5. [Predict into the Future](#predict-into-the-future)

The ML model instantiated above (with the best hyper-parameters) will now be trained on all the available data (excluding a trailing gap) and then used to predict into the future

<a id="read-in-all-data"></a>

### 5.1. [Read in all Data](#read-in-all-data)

In [ ]:
%%time
data = az_load_data(
    blob_dict_inputs,
    az_storage_container_name,
    ["date"],
    ["train"],
)[0]

**Note**
1. This is a repeat of section [2.1](#read-data).

<a id="sort-all-data"></a>

### 5.2. [Sort all Data](#sort-all-data)

In [ ]:
%%time
multi_col_sort_pipe = Pipeline(
    [
        ("sisort", ct.DFMultiColSort(["store", "item", "date"], [True, True, True])),
    ]
)
data = multi_col_sort_pipe.fit_transform(data)

**Note**
1. This is a repeat of section [2.2](#read-data).

<a id="create-usable-data-by-dropping-observations-in-trailing-gap-from-all-data"></a>

### 5.3. [Create Usable Data by Dropping Observations in Trailing Gap from all Data](#create-usable-data-by-dropping-observations-in-trailing-gap-from-all-data)

In [ ]:
%%time
train_end_date = data["date"].max() - pd.DateOffset(days=GAP)
data = data[data["date"] <= train_end_date]

**Note**
1. This is a repeat of section [2.3](#create-usable-data-by-dropping-observations-in-trailing-gap).

<a id="set-multi-index-to-support-cv-splitter-for-all-data"></a>

### 5.4. [Set Multi-Index to Support CV Splitter for all Data](#set-multi-index-to-support-cv-splitter-for-all-data)

In [ ]:
%%time
data = data.assign(
    symbol=data["store"].astype(str) + "_" + data["item"].astype(str)
).set_index(["symbol", "date"])
display(data.head().append(data.tail()))

**Note**
1. This is a repeat of section [2.5](#set-multi-index-to-support-cv-splitter).

<a id="make-predictions"></a>

### 5.5. [Make Predictions](#make-predictions)

In [ ]:
%%time
(
    df_pred_test,
    trained_model,
    train_end_date_manual,
    test_start_date_manual,
    test_end_date_manual,
) = predict_future(
    0,
    data,
    lags,
    window_size,
    used_columns,
    categ_fea,
    FIRST_DATE,
    GAP,
    HORIZON,
    pd.DataFrame(),
    best_lgb_params,
    lgbm_smape,
    lgb_fit_params,
)
assert len(df_pred_test.index.get_level_values(1).unique()) == GAP
inference_start_date = train_end_date + pd.DateOffset(days=GAP + 1)
inference_end_date = inference_start_date + pd.DateOffset(days=GAP - 1)
inference_dates = pd.date_range(inference_start_date, inference_end_date)
assert df_pred_test.index.get_level_values(1).unique().equals(inference_dates)
assert train_end_date_manual == train_end_date
assert test_start_date_manual == inference_dates.min().strftime("%Y-%m-%d")
assert test_end_date_manual == inference_dates.max().strftime("%Y-%m-%d")

<a id="notes"></a>

## 6. [Notes](#notes)

1.  Change cross-validation
    -   Increase number of cross-validation folds
        -   currently, (to reduce computation duration) this is only done with three folds
2.  Insignificant differences were observed when using `LightGBM` with and without normalizing/standardizing numerical features, which is not surprising since tree-based models are not distance based and so can handle features on varying scales
3.  Since `LightGBM` can handle categoricals (without manually encoding them), explore this direct use of categorical features during ML model development
    -   set each (`store` and `item`) of the columns to [an `int` datatype](https://lightgbm.readthedocs.io/en/latest/Python-Intro.html), which can be done by label-encoding them (if they are strings) and specify a list of categorical features to [LightGBM's `categorical_feature` hyperparameter](https://lightgbm.readthedocs.io/en/latest/Parameters.html#categorical_feature) to `.train()` for the learning API ([link](https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.train.html#lightgbm.train)) or `.fit()` for the `sklearn` API ([link](https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMRegressor.html#lightgbm.LGBMRegressor.fit)).

---

<span style="float:left">
    2021 | <a href="https://github.com/edesz/store-item-forecast">@edesz</a> (MIT)
</span>